In [ ]:
from bayes3d.dino import Dino
import bayes3d as b
import os
import torch
import numpy as np
import matplotlib
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import torchvision.transforms as T
import bayes3d.ycb_loader
import bayes3d.o3d_viz
from tqdm import tqdm
import open3d as o3d


In [ ]:
!pip install --upgrade diffusers[torch]
!pip install transformers
!pip install -U xformers
!pip install ipympl
!pip install timm
!pip install torch==1.12.0

In [ ]:
%matplotlib widget
import os
import gc
import random
import json
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
from torchvision.transforms import PILToTensor
from typing import Any, Callable, Dict, List, Optional, Union, Tuple
from diffusers.models.unet_2d_condition import UNet2DConditionModel
from diffusers import DDIMScheduler
from diffusers import StableDiffusionPipeline

In [ ]:
b.setup_visualizer()

In [ ]:
class MyUNet2DConditionModel(UNet2DConditionModel):
    def forward(
        self,
        sample: torch.FloatTensor,
        timestep: Union[torch.Tensor, float, int],
        up_ft_indices,
        encoder_hidden_states: torch.Tensor,
        class_labels: Optional[torch.Tensor] = None,
        timestep_cond: Optional[torch.Tensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        cross_attention_kwargs: Optional[Dict[str, Any]] = None):
        r"""
        Args:
            sample (`torch.FloatTensor`): (batch, channel, height, width) noisy inputs tensor
            timestep (`torch.FloatTensor` or `float` or `int`): (batch) timesteps
            encoder_hidden_states (`torch.FloatTensor`): (batch, sequence_length, feature_dim) encoder hidden states
            cross_attention_kwargs (`dict`, *optional*):
                A kwargs dictionary that if specified is passed along to the `AttnProcessor` as defined under
                `self.processor` in
                [diffusers.cross_attention](https://github.com/huggingface/diffusers/blob/main/src/diffusers/models/cross_attention.py).
        """
        # By default samples have to be AT least a multiple of the overall upsampling factor.
        # The overall upsampling factor is equal to 2 ** (# num of upsampling layears).
        # However, the upsampling interpolation output size can be forced to fit any upsampling size
        # on the fly if necessary.
        default_overall_up_factor = 2**self.num_upsamplers

        # upsample size should be forwarded when sample is not a multiple of `default_overall_up_factor`
        forward_upsample_size = False
        upsample_size = None

        if any(s % default_overall_up_factor != 0 for s in sample.shape[-2:]):
            # logger.info("Forward upsample size to force interpolation output size.")
            forward_upsample_size = True

        # prepare attention_mask
        if attention_mask is not None:
            attention_mask = (1 - attention_mask.to(sample.dtype)) * -10000.0
            attention_mask = attention_mask.unsqueeze(1)

        # 0. center input if necessary
        if self.config.center_input_sample:
            sample = 2 * sample - 1.0

        # 1. time
        timesteps = timestep
        if not torch.is_tensor(timesteps):
            # TODO: this requires sync between CPU and GPU. So try to pass timesteps as tensors if you can
            # This would be a good case for the `match` statement (Python 3.10+)
            is_mps = sample.device.type == "mps"
            if isinstance(timestep, float):
                dtype = torch.float32 if is_mps else torch.float64
            else:
                dtype = torch.int32 if is_mps else torch.int64
            timesteps = torch.tensor([timesteps], dtype=dtype, device=sample.device)
        elif len(timesteps.shape) == 0:
            timesteps = timesteps[None].to(sample.device)

        # broadcast to batch dimension in a way that's compatible with ONNX/Core ML
        timesteps = timesteps.expand(sample.shape[0])

        t_emb = self.time_proj(timesteps)

        # timesteps does not contain any weights and will always return f32 tensors
        # but time_embedding might actually be running in fp16. so we need to cast here.
        # there might be better ways to encapsulate this.
        t_emb = t_emb.to(dtype=self.dtype)

        emb = self.time_embedding(t_emb, timestep_cond)

        if self.class_embedding is not None:
            if class_labels is None:
                raise ValueError("class_labels should be provided when num_class_embeds > 0")

            if self.config.class_embed_type == "timestep":
                class_labels = self.time_proj(class_labels)

            class_emb = self.class_embedding(class_labels).to(dtype=self.dtype)
            emb = emb + class_emb

        # 2. pre-process
        sample = self.conv_in(sample)

        # 3. down
        down_block_res_samples = (sample,)
        for downsample_block in self.down_blocks:
            if hasattr(downsample_block, "has_cross_attention") and downsample_block.has_cross_attention:
                sample, res_samples = downsample_block(
                    hidden_states=sample,
                    temb=emb,
                    encoder_hidden_states=encoder_hidden_states,
                    attention_mask=attention_mask,
                    cross_attention_kwargs=cross_attention_kwargs,
                )
            else:
                sample, res_samples = downsample_block(hidden_states=sample, temb=emb)

            down_block_res_samples += res_samples

        # 4. mid
        if self.mid_block is not None:
            sample = self.mid_block(
                sample,
                emb,
                encoder_hidden_states=encoder_hidden_states,
                attention_mask=attention_mask,
                cross_attention_kwargs=cross_attention_kwargs,
            )

        # 5. up
        up_ft = {}
        for i, upsample_block in enumerate(self.up_blocks):

            if i > np.max(up_ft_indices):
                break

            is_final_block = i == len(self.up_blocks) - 1

            res_samples = down_block_res_samples[-len(upsample_block.resnets) :]
            down_block_res_samples = down_block_res_samples[: -len(upsample_block.resnets)]

            # if we have not reached the final block and need to forward the
            # upsample size, we do it here
            if not is_final_block and forward_upsample_size:
                upsample_size = down_block_res_samples[-1].shape[2:]

            if hasattr(upsample_block, "has_cross_attention") and upsample_block.has_cross_attention:
                sample = upsample_block(
                    hidden_states=sample,
                    temb=emb,
                    res_hidden_states_tuple=res_samples,
                    encoder_hidden_states=encoder_hidden_states,
                    cross_attention_kwargs=cross_attention_kwargs,
                    upsample_size=upsample_size,
                    attention_mask=attention_mask,
                )
            else:
                sample = upsample_block(
                    hidden_states=sample, temb=emb, res_hidden_states_tuple=res_samples, upsample_size=upsample_size
                )

            if i in up_ft_indices:
                up_ft[i] = sample.detach()

        output = {}
        output['up_ft'] = up_ft
        return output

class OneStepSDPipeline(StableDiffusionPipeline):
    @torch.no_grad()
    def __call__(
        self,
        img_tensor,
        t,
        up_ft_indices,
        negative_prompt: Optional[Union[str, List[str]]] = None,
        generator: Optional[Union[torch.Generator, List[torch.Generator]]] = None,
        prompt_embeds: Optional[torch.FloatTensor] = None,
        callback: Optional[Callable[[int, int, torch.FloatTensor], None]] = None,
        callback_steps: int = 1,
        cross_attention_kwargs: Optional[Dict[str, Any]] = None
    ):

        device = self._execution_device
        latents = self.vae.encode(img_tensor).latent_dist.sample() * self.vae.config.scaling_factor
        t = torch.tensor(t, dtype=torch.long, device=device)
        noise = torch.randn_like(latents).to(device)
        latents_noisy = self.scheduler.add_noise(latents, noise, t)
        unet_output = self.unet(latents_noisy,
                               t,
                               up_ft_indices,
                               encoder_hidden_states=prompt_embeds,
                               cross_attention_kwargs=cross_attention_kwargs)
        return unet_output


class SDFeaturizer:
    def __init__(self, sd_id='stabilityai/stable-diffusion-2-1'):
        unet = MyUNet2DConditionModel.from_pretrained(sd_id, subfolder="unet")
        onestep_pipe = OneStepSDPipeline.from_pretrained(sd_id, unet=unet, safety_checker=None)
        onestep_pipe.vae.decoder = None
        onestep_pipe.scheduler = DDIMScheduler.from_pretrained(sd_id, subfolder="scheduler")
        gc.collect()
        onestep_pipe = onestep_pipe.to("cuda")
        onestep_pipe.enable_attention_slicing()
        onestep_pipe.enable_xformers_memory_efficient_attention()
        self.pipe = onestep_pipe

    @torch.no_grad()
    def forward(self,
                img_tensor, # single image, [1,c,h,w]
                prompt,
                t=261,
                up_ft_index=1,
                ensemble_size=8):
        img_tensor = img_tensor.repeat(ensemble_size, 1, 1, 1).cuda() # ensem, c, h, w
        prompt_embeds = self.pipe._encode_prompt(
            prompt=prompt,
            device='cuda',
            num_images_per_prompt=1,
            do_classifier_free_guidance=False) # [1, 77, dim]
        prompt_embeds = prompt_embeds.repeat(ensemble_size, 1, 1)
        unet_ft_all = self.pipe(
            img_tensor=img_tensor,
            t=t,
            up_ft_indices=[up_ft_index],
            prompt_embeds=prompt_embeds)
        unet_ft = unet_ft_all['up_ft'][up_ft_index] # ensem, c, h, w
        unet_ft = unet_ft.mean(0, keepdim=True) # 1,c,h,w
        return unet_ft

In [ ]:
bop_ycb_dir = os.path.join(b.utils.get_assets_dir(), "bop/ycbv")
rgbd, gt_ids, gt_poses, masks = b.ycb_loader.get_test_img('49', '1', bop_ycb_dir)
img1 = b.get_rgb_image(rgbd.rgb)
img1

In [ ]:
bop_ycb_dir = os.path.join(b.utils.get_assets_dir(), "bop/ycbv")
rgbd, gt_ids, gt_poses, masks = b.ycb_loader.get_test_img('51', '1', bop_ycb_dir)
img2 = b.get_rgb_image(rgbd.rgb)
img2

In [ ]:
dift = SDFeaturizer(sd_id='stabilityai/stable-diffusion-2-1')

In [ ]:
img1

In [ ]:
def get_embeddings(img):
    img = img.convert('RGB')
    img_tensor = (PILToTensor()(img) / 255.0 - 0.5) * 2
    output_tensor = dift.forward(img_tensor, prompt="object", ensemble_size=2)
    output = jnp.transpose(jnp.array(output_tensor.cpu().detach().numpy())[0], (1,2,0))
    del img_tensor
    del output_tensor
    torch.cuda.empty_cache()
    return output


def get_embeddings_from_rgbd(rgbd):
    img = b.get_rgb_image(rgbd.rgb).convert('RGB')
    return get_embeddings(img)


embeddings1 = jax.image.resize(get_embeddings(img1), (img1.height, img1.width, embeddings.shape[-1]), "bilinear")
embeddings2 = jax.image.resize(get_embeddings(img2), (img1.height, img1.width, embeddings.shape[-1]), "bilinear")


In [ ]:
class Demo:
    def __init__(self, imgs, ft, img_size):
        self.ft = ft # NCHW
        self.imgs = imgs
        self.num_imgs = len(imgs)
        self.img_size = img_size

    def plot_img_pairs(self, fig_size=3, alpha=0.45, scatter_size=70):

        fig, axes = plt.subplots(1, self.num_imgs, figsize=(fig_size*self.num_imgs, fig_size))

        plt.tight_layout()

        for i in range(self.num_imgs):
            axes[i].imshow(self.imgs[i])
            axes[i].axis('off')
            if i == 0:
                axes[i].set_title('source image')
            else:
                axes[i].set_title('target image')

        num_channel = self.ft.size(1)
        cos = nn.CosineSimilarity(dim=1)

        def onclick(event):
            if event.inaxes == axes[0]:
                with torch.no_grad():

                    x, y = int(np.round(event.xdata)), int(np.round(event.ydata))

                    src_ft = self.ft[0].unsqueeze(0)
                    src_ft = nn.Upsample(size=(self.img_size, self.img_size), mode='bilinear')(src_ft)
                    src_vec = src_ft[0, :, y, x].view(1, num_channel, 1, 1)  # 1, C, 1, 1
                    del src_ft
                    gc.collect()
                    torch.cuda.empty_cache()

                    trg_ft = nn.Upsample(size=(self.img_size, self.img_size), mode='bilinear')(self.ft[1:])
                    cos_map = cos(src_vec, trg_ft).cpu().numpy()  # N, H, W
                    
                    del trg_ft
                    gc.collect()
                    torch.cuda.empty_cache()

                    axes[0].clear()
                    axes[0].imshow(self.imgs[0])
                    axes[0].axis('off')
                    axes[0].scatter(x, y, c='r', s=scatter_size)
                    axes[0].set_title('source image')

                    for i in range(1, self.num_imgs):
                        max_yx = np.unravel_index(cos_map[i-1].argmax(), cos_map[i-1].shape)
                        axes[i].clear()

                        heatmap = cos_map[i-1]
                        heatmap = (heatmap - np.min(heatmap)) / (np.max(heatmap) - np.min(heatmap))  # Normalize to [0, 1]
                        axes[i].imshow(self.imgs[i])
                        axes[i].imshow(255 * heatmap, alpha=alpha, cmap='viridis')
                        axes[i].axis('off')
                        axes[i].scatter(max_yx[1].item(), max_yx[0].item(), c='r', s=scatter_size)
                        axes[i].set_title('target image')

                    del cos_map
                    del heatmap
                    gc.collect()

        fig.canvas.mpl_connect('button_press_event', onclick)
        plt.show()

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt


In [ ]:
scaling_factor= o.shape[0] / training_images[0].rgb.shape[0]
scaled_down_intrinsics = b.camera.scale_camera_parameters(intrinsics, scaling_factor)
scaled_down_intrinsics

In [ ]:
num_images = len(training_images)
training_indices = jnp.arange(0,num_images-1, num_images//4)

keypoint_coordinates = []
keypoint_embeddings = []

for idx in tqdm(training_indices):
    angle = training_angles[idx]
    training_image = training_images[idx]
    pose = b.t3d.inverse_pose(b.t3d.transform_from_pos_target_up(
        jnp.array([0.0, 0.6, 0.0]),
        jnp.array([0.0, 0.0, 0.0]),
        jnp.array([0.0, 0.0, 1.0]),
    )) @ b.t3d.transform_from_axis_angle(jnp.array([0.0, 0.0, 1.0]), angle)

    scaled_down_training_image = b.scale_rgbd(training_image, scaling_factor)
    embeddings = get_embeddings(training_image)
    # del embeddings
    foreground_mask = (jnp.inf != scaled_down_training_image.depth)
    foreground_pixel_coordinates = jnp.transpose(jnp.vstack(jnp.where(foreground_mask)))

    NUM_KEYPOINTS_TO_SELECT = jnp.min(jnp.array([2000,foreground_pixel_coordinates.shape[0]]))
    subset = jax.random.choice(jax.random.PRNGKey(10),foreground_pixel_coordinates.shape[0], shape=(NUM_KEYPOINTS_TO_SELECT,), replace=False)

    depth = jnp.array(scaled_down_training_image.depth)
    depth = depth.at[depth == jnp.inf].set(0.0)
    point_cloud_image = b.t3d.unproject_depth(depth, scaled_down_training_image.intrinsics)

    keypoint_world_coordinates = point_cloud_image[foreground_pixel_coordinates[subset,0], foreground_pixel_coordinates[subset,1],:]
    _keypoint_coordinates = b.t3d.apply_transform(keypoint_world_coordinates, b.t3d.inverse_pose(pose))
    _keypoint_embeddings = embeddings[foreground_pixel_coordinates[subset,0], foreground_pixel_coordinates[subset,1],:]
    
    keypoint_coordinates.append(_keypoint_coordinates)
    keypoint_embeddings.append(_keypoint_embeddings)
    del embeddings

    

In [ ]:
keypoint_coordinates = jnp.concatenate(keypoint_coordinates)
keypoint_embeddings = jnp.concatenate(keypoint_embeddings)
print(keypoint_coordinates.shape)
print(keypoint_embeddings.shape)

In [ ]:
b.setup_renderer(scaled_down_training_image.intrinsics)
b.RENDERER.add_mesh_from_file(mesh_filename, scaling_factor=SCALING_FACTOR)
b.setup_renderer(scaled_down_training_image.intrinsics)
b.RENDERER.add_mesh_from_file(mesh_filename, scaling_factor=SCALING_FACTOR)

In [ ]:


def render_embedding_image(pose, keypoint_coordinates, keypoint_embeddings):
    point_cloud_img = b.RENDERER.render_single_object(pose, jnp.int32(0))[:,:,:3]
    point_cloud_img_in_object_frame = b.t3d.apply_transform(point_cloud_img, b.t3d.inverse_pose(pose))

    distances_to_keypoints = (
        jnp.linalg.norm(point_cloud_img_in_object_frame[:, :,None,...] - keypoint_coordinates[None, None,:,...],
        axis=-1
    ))
    index_of_nearest_keypoint = distances_to_keypoints.argmin(2)
    distance_to_nearest_keypoints = distances_to_keypoints.min(2)

    DISTANCE_THRESHOLD = 0.04
    valid_match_mask = (distance_to_nearest_keypoints < DISTANCE_THRESHOLD)[...,None]
    selected_keypoints = keypoint_coordinates[index_of_nearest_keypoint]
    rendered_embeddings_image = keypoint_embeddings[index_of_nearest_keypoint] * valid_match_mask
    return point_cloud_img, rendered_embeddings_image

def score_pose(pose, keypoint_coordinates, keypoint_embeddings, observed_embeddings):
    _,rendered_embedding_image = render_embedding_image(pose, keypoint_coordinates, keypoint_embeddings)
    dot_products = jnp.einsum("abi,abi->ab", rendered_embedding_image, observed_embeddings)
    return dot_products.mean()

def get_pca(embeddings):
    features_flat = torch.from_numpy(np.array(embeddings).reshape(-1, embeddings.shape[-1]))
    U, S, V = torch.pca_lowrank(features_flat - features_flat.mean(0), niter=10)
    proj_PCA = jnp.array(V[:, :3])
    return proj_PCA

def get_colors(features, proj_V):
    features_flat = features.reshape(-1, features.shape[-1])
    feat_rgb = features_flat @ proj_V
    feat_rgb = (feat_rgb + 1.0) / 2.0
    feat_rgb = feat_rgb.reshape(features.shape[:-1] + (3,))
    return feat_rgb

score_pose_jit = jax.jit(score_pose)

In [ ]:

angles = jnp.linspace(-jnp.pi, jnp.pi, 300)
angle_to_pose = lambda angle : b.t3d.inverse_pose(b.t3d.transform_from_pos_target_up(
        jnp.array([0.0, 0.6, 0.0]),
        jnp.array([0.0, 0.0, 0.0]),
        jnp.array([0.0, 0.0, 1.0]),
    )) @ b.t3d.transform_from_axis_angle(jnp.array([0.0, 0.0, 1.0]), angle)
scorer = lambda angle, observed_embeddings: score_pose(
    angle_to_pose(angle),
    keypoint_coordinates, keypoint_embeddings, observed_embeddings
)
scorer_jit = jax.jit(scorer)
scorer_parallel_jit = jax.jit(jax.vmap(scorer, in_axes=(0,None)))


In [ ]:
proj_V = get_pca(keypoint_embeddings)

In [ ]:
IDX = 3
test_rgbd = training_images[IDX]
observed_embeddings = get_embeddings(test_rgbd)
training_angles[IDX]

In [ ]:
posterior = jnp.concatenate([scorer_parallel_jit(i, observed_embeddings) for i in jnp.array_split(angles, 10)])
best_angle = angles[posterior.argmax()]
print(best_angle)
best_pose = angle_to_pose(best_angle)

In [ ]:
colors = get_colors(observed_embeddings, proj_V)
embedding_image = b.scale_image(b.get_rgb_image(colors * 255.0),14.0)
embedding_image

In [ ]:
pc_img, img = render_embedding_image(angle_to_pose(best_angle), keypoint_coordinates, keypoint_embeddings)
colors = get_colors(observed_embeddings, proj_V)
rgba = jnp.array(b.get_rgb_image(colors * 255.0))
rgba = rgba.at[pc_img[:,:,2] > intrinsics.far - 0.01, :3].set(255.0)
rerendered_embeddings = b.scale_image(b.get_rgb_image(rgba),14.0)
rerendered_embeddings

In [ ]:
b.multi_panel([
    b.get_rgb_image(test_rgbd.rgb),
    embedding_image,
    rerendered_embeddings
]
             )

In [ ]:
import meshcat.geometry as g

proj_V = get_pca(keypoint_embeddings)
colors = get_colors(keypoint_embeddings, proj_V)
b.clear()
obj = g.PointCloud(np.transpose(keypoint_coordinates)*30.0, np.transpose(colors), size=0.1)
b.meshcatviz.VISUALIZER["2"].set_object(obj)

In [ ]:
b.setup_renderer(test_rgbd.intrinsics)
# b.RENDERER.add_mesh_from_file(mesh_filename, scaling_factor=SCALING_FACTOR)
point_cloud_img = b.RENDERER.render_single_object(best_pose, jnp.int32(0))[:,:,:3]

In [ ]:
mask = (test_rgbd.intrinsics.near < point_cloud_img[:,:,2]) * (point_cloud_img[:,:,2] < test_rgbd.intrinsics.far)
print(point_cloud_img[:,:,2][mask].min(), point_cloud_img[:,:,2][mask].max())
b.get_depth_image(1.0 * mask)
img = jnp.array(b.get_depth_image(point_cloud_img[:,:,2], min=0.46, max=0.65))
img = img.at[jnp.invert(mask) , :3].set(255.0)
b.get_rgb_image(img)

In [ ]:
b.overlay_image(b.get_rgb_image(test_rgbd.rgb), b.get_rgb_image(img), 0.7)

In [ ]:
pose = best_pose
point_cloud_img = b.RENDERER.render_single_object(pose, jnp.int32(0))[:,:,:3]
point_cloud_img_in_object_frame = b.t3d.apply_transform(point_cloud_img, b.t3d.inverse_pose(pose))

distances_to_keypoints = (
    jnp.linalg.norm(point_cloud_img_in_object_frame[:, :,None,...] - keypoint_coordinates[None, None,:,...],
    axis=-1
))
index_of_nearest_keypoint = distances_to_keypoints.argmin(2)
distance_to_nearest_keypoints = distances_to_keypoints.min(2)

DISTANCE_THRESHOLD = 0.2
valid_match_mask = (distance_to_nearest_keypoints < DISTANCE_THRESHOLD)[...,None]
selected_keypoints = keypoint_coordinates[index_of_nearest_keypoint]
rendered_embeddings_image = keypoint_embeddings[index_of_nearest_keypoint] * valid_match_mask

colors = get_colors(rendered_embeddings_image, proj_V)
b.get_rgb_image(colors * 255.0)